Read files

In [4]:
import numpy as np
import random

def read_vrp_file(filepath: str):
    data = {
        "name": None,
        "dimension": None,
        "capacity": None,
        "customers": {},   # now contains x,y,demand
        "depot": None,
        "distance_matrix": None
    }

    coordinates = {}
    demands = {}

    section = None
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("EOF"):
                continue

            # Metadata
            if line.startswith("NAME"):
                data["name"] = line.split(":")[1].strip()
            elif line.startswith("DIMENSION"):
                data["dimension"] = int(line.split(":")[1].strip())
            elif line.startswith("CAPACITY"):
                data["capacity"] = int(line.split(":")[1].strip())
            elif line.startswith("EDGE_WEIGHT_TYPE"):
                data["edge_weight_type"] = line.split(":")[1].strip()

            # Sections
            elif line.startswith("NODE_COORD_SECTION"):
                section = "coords"
                continue
            elif line.startswith("DEMAND_SECTION"):
                section = "demand"
                continue
            elif line.startswith("DEPOT_SECTION"):
                section = "depot"
                continue

            # Parse sections
            elif section == "coords":
                parts = line.split()
                node, x, y = int(parts[0]), float(parts[1]), float(parts[2])
                coordinates[node] = (x, y)

            elif section == "demand":
                parts = line.split()
                node, demand = int(parts[0]), int(parts[1])
                demands[node] = demand

            elif section == "depot":
                node = int(line)
                if node == -1:  # End of depot section
                    section = None
                else:
                    data["depot"] = node

    # Merge coordinates and demands into customers
    for node, (x, y) in coordinates.items():
        demand = demands.get(node, 0)
        data["customers"][node] = (x, y, demand)

    return data


data = read_vrp_file("content/A-n32-k5.vrp")
data['customers']

{1: (82.0, 76.0, 0),
 2: (96.0, 44.0, 19),
 3: (50.0, 5.0, 21),
 4: (49.0, 8.0, 6),
 5: (13.0, 7.0, 19),
 6: (29.0, 89.0, 7),
 7: (58.0, 30.0, 12),
 8: (84.0, 39.0, 16),
 9: (14.0, 24.0, 6),
 10: (2.0, 39.0, 16),
 11: (3.0, 82.0, 8),
 12: (5.0, 10.0, 14),
 13: (98.0, 52.0, 21),
 14: (84.0, 25.0, 16),
 15: (61.0, 59.0, 3),
 16: (1.0, 65.0, 22),
 17: (88.0, 51.0, 18),
 18: (91.0, 2.0, 19),
 19: (19.0, 32.0, 1),
 20: (93.0, 3.0, 24),
 21: (50.0, 93.0, 8),
 22: (98.0, 14.0, 12),
 23: (5.0, 42.0, 4),
 24: (42.0, 9.0, 8),
 25: (61.0, 62.0, 24),
 26: (9.0, 97.0, 24),
 27: (80.0, 55.0, 2),
 28: (57.0, 69.0, 20),
 29: (23.0, 15.0, 15),
 30: (20.0, 70.0, 2),
 31: (85.0, 60.0, 14),
 32: (98.0, 5.0, 9)}

Creates individuals

In [5]:
customers = [cust for key, cust in data['customers'].items()]
print(customers)
vehicleCapacity = data['capacity']
depot = data['depot']-1

# hardcoding vehicles, not good we need change

def vehicleCheck(customer, vehicle):
  usedSpace = 0
  for cust in vehicle:
    usedSpace += customers[cust][2]
  return usedSpace + customer[2] <= vehicleCapacity

def create_individual(vehicles):
    # Step 2: Remove one random element
    individual = [[] for _ in range(vehicles)]

    for idx in range(1,len(customers)):
        customer = customers[idx]
        chosen_vehicle = random.randint(0, vehicles - 1)
        if vehicleCheck(customer, individual[chosen_vehicle]):
            individual[chosen_vehicle].append(idx)
        else:
            individual.append([idx])

    return individual

create_individual(5)

[(82.0, 76.0, 0), (96.0, 44.0, 19), (50.0, 5.0, 21), (49.0, 8.0, 6), (13.0, 7.0, 19), (29.0, 89.0, 7), (58.0, 30.0, 12), (84.0, 39.0, 16), (14.0, 24.0, 6), (2.0, 39.0, 16), (3.0, 82.0, 8), (5.0, 10.0, 14), (98.0, 52.0, 21), (84.0, 25.0, 16), (61.0, 59.0, 3), (1.0, 65.0, 22), (88.0, 51.0, 18), (91.0, 2.0, 19), (19.0, 32.0, 1), (93.0, 3.0, 24), (50.0, 93.0, 8), (98.0, 14.0, 12), (5.0, 42.0, 4), (42.0, 9.0, 8), (61.0, 62.0, 24), (9.0, 97.0, 24), (80.0, 55.0, 2), (57.0, 69.0, 20), (23.0, 15.0, 15), (20.0, 70.0, 2), (85.0, 60.0, 14), (98.0, 5.0, 9)]


[[7, 13, 17, 21],
 [4, 10, 15, 19, 30],
 [5, 18, 20, 22, 26, 28],
 [2, 3, 8, 11, 14, 25, 27],
 [1, 6, 9, 12, 16, 23, 29],
 [24],
 [31]]